# Homework 1
Student Name: Huynh Quang Nguyen Vo

CWID: A20446163

In [16]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

##
# Initial 
#
months = np.linspace(1, 6, 6)
demands = [1055, 700, 1645, 1600, 1095, 845]
production_capacities = [1000, 300]
production_costs = [10, 15]
inventory_cost = 2
backordered_cost = 5
inventories = 0 
backorders = 0 

indices = range(1,len(demands)+1)

try:
    
    m = gp.Model('ProductionPlanning')
    
    #vector of variables
    x = m.addVars(indices, lb=0.0, vtype = GRB.INTEGER, name='units_produced-rt')
    y = m.addVars(indices, lb=0.0, vtype = GRB.INTEGER, name='units_produced_ot')
    z = m.addVars(indices, lb=0.0, vtype = GRB.INTEGER, name='units_carried')
    w = m.addVars(indices, lb=0.0, vtype = GRB.INTEGER, name='back_order_units')
    
    
    #Set objective
    m.setObjective(gp.quicksum([production_costs[0] * x[i] + production_costs[1] * y[i] + inventory_cost * z[i]
                             + backordered_cost * w[i] for j in indices]), GRB.MINIMIZE)
    
    #adding constraints
    m.addConstr(z[1] == inventories)
    m.addConstr(w[1] == backorders)
    m.addConstr(z[6] == inventories)
    m.addConstr(w[6] == backorders)
    
    for i in indices:
        if i == 1:
            m.addConstr(x[i] + y[i]- z[i] == demands[i-1] - w[i])
        else:
            m.addConstr(x[i] + y[i] + z[i-1] - z[i] == demands[i-1] + w[i-1] - w[i])
    
    for i in indices:
        m.addConstr(x[i] <= production_capacities[0])
    
        m.addConstr(y[i] <= production_capacities[1])
    
    #write in LP format
    m.write('ProductionPlanning.lp')
    m.optimize()
    
    #output
    if m.status == GRB.OPTIMAL:
        for v in m.getVars():
            print('%s %g' % (v.varName, v.x))
            print('Obj: %g' % m.objVal)
    elif m.status == GRB.INFEASIBLE:
        print('LP is infeasible.')
    elif m.status == GRB.UNBOUNDED:
        print('LP is unbounded.')
except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ": " + str(e))
except AttributeError:
        print('Encountered an attribute error') 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 22 rows, 24 columns and 50 nonzeros
Model fingerprint: 0xf3d79514
Variable types: 0 continuous, 24 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 2e+03]
Found heuristic solution: objective 87000.000000
Presolve removed 22 rows and 24 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 2: 41100 87000 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.110000000000e+04, best bound 4.110000000000e+04, gap 0.0000%
units_produced-rt[1] 755
Obj: 41100
units_produced-rt[